## Кадры решают всё (но это не точно)

### Кадры, да...

In [114]:
%%time

# импорт библиотек
import pandas as pd
import numpy as np
import os
import sys
import shutil  # упаковка в архив
import warnings

warnings.simplefilter(action="ignore")

CPU times: total: 0 ns
Wall time: 0 ns


In [115]:
# функции для автоматической замены названий регионов, имеющих различное написание

def words(string):
    """возвращает список значимых слов в названии субъекта,
    приведённых к нижнему регистру"""
    str = ""
    tp_list = ["республика", "край", "область", "автономный", "автономная", "округ", "народная", "г"]
    for symbol in string:
        symbol = symbol if symbol.isalpha() else "$"
        str = str + symbol
    lst = str.split("$")
    while "" in lst:
        lst.remove("")
    lst = [element.lower() for element in lst]
    for w in tp_list:
        while w in lst:
            lst.remove(w)
    return lst

def letter(word_1, word_2):
    """побуквенное сравнение слов. Сравнивает первую букву одного слова с 
    первой буковй второрого слова, вторую букву первого слова со второй буквой
    второго слова и т.д. Возвращает долю совпавших букв в слове с наименьшим числом символов"""
    k = 0
    for i in range(min(len(word_1), len(word_2)) - 1):
        if (word_1[i].lower() == word_2[i].lower()) and (word_1[i+1].lower() == word_2[i+1].lower()):
            k += 1
    return k/min(len(word_1), len(word_2))

def letter_part(word_1, word_2):
    """побуквенное сравнение частей слов. Ищет 3 первые буквы наиболее короткого слова
    (подряд) во втором (более длинном или равной длины) слове, если находит - ищет 4 первые буквы первого слова
    и т.д. Возвращает долю совпавших букв в слове с наименьшим числом символов"""
    str_0 = word_2 if len(word_1) > len(word_2) else word_1
    str_1 = word_1 if str_0 == word_2 else word_2
    poz = 3
    while str_0.lower()[0:poz] in str_1.lower():
        poz += 1
    return (poz - 1)/len(str_0)

def real_region_name (region_name_1, region_name_2):
    """возвращает "правильное" название региона с использованием letter"""
    lst_1 = words(region_name_1)
    lst_2 = words(region_name_2)
    noret = True
    real_name = region_name_1 if len(region_name_1) > len(region_name_2) else region_name_2
    for element in lst_1:
        if element in lst_2:
            return real_name
            noret = False
    if noret:
        for element_1 in lst_1:
            for element_2 in lst_2:
                if letter(element_1, element_2) >= 0.7:
                    return real_name
                
def real_region_name_part (region_name_1, region_name_2):
    """возвращает "правильное" название региона с использованием letter_part"""
    lst_1 = words(region_name_1)
    lst_2 = words(region_name_2)
    noret = True
    real_name = region_name_1 if len(region_name_1) > len(region_name_2) else region_name_2
    for element in lst_1:
        if element in lst_2:
            return real_name
            noret = False
    if noret:
        for element_1 in lst_1:
            for element_2 in lst_2:
                if letter_part(element_1, element_2) >= 0.7:
                    return real_name
                
def region_name_change (region_name, table_change):
    """заменяет названия регионов на названия из списка замены"""
    try:
        dict_change = table_change.set_index(table_change.columns[0]).T.to_dict("records")[0]
    except Exception:
        dict_change = table_change.set_index(table_change.columns[0]).T.to_dict("records")
    try:
        if region_name in dict_change.keys():
            return dict_change[region_name]
        else:
            return region_name
    except Exception:
        return region_name

#### Загрузка данных конфигурации

In [127]:
%%time
# путь к файлу конфигурации
config_path = "C:/personal_auto/config.xlsx"

# загрузка таблицы соответствия названий регионов из файла конфигурации
df_reg_ref = pd.read_excel(
    config_path,
    sheet_name="regions",
    index_col=None,
    dtype="str",
)

# загрузка групп специальностей из файла конфигурации
df_spec_groups = pd.read_excel(
    config_path,
    sheet_name="spec",
    index_col=None,
    dtype="str",
)
# группы специальностей для фильтрации
df_spec_groups_f = df_spec_groups.dropna().reset_index(drop=True)
# группы специальностей для расшифровки состава групп
df_spec_groups_merge = df_spec_groups_f.groupby("group", as_index=False).agg(
    {"spec_name": ", ".join}
)


# загрузка типа совместительства из файла конфигурации
df_cross_config = pd.read_excel(
    config_path,
    sheet_name="cross",
    index_col=None,
    dtype="str",
)
# типы совместительства для фильрации
df_cross_config_f = df_cross_config.dropna().reset_index(drop=True)


# загрузка типа временного неисполнения из файла конфигурации
df_tmp_config = pd.read_excel(
    config_path,
    sheet_name="tmp",
    index_col=None,
    dtype="str",
)
# типы временного неисполнения для фильрации
df_tmp_config_f = df_tmp_config.dropna().reset_index(drop=True)


# загрузка типов ведомственной принадлежности из файла конфигурации
df_dpt_config = pd.read_excel(
    config_path,
    sheet_name="dpt",
    index_col=None,
    dtype="str",
)
# ведомственная принадлежность для фильрации
df_dpt_config_f = df_dpt_config.dropna().reset_index(drop=True)

CPU times: total: 31.2 ms
Wall time: 96.4 ms


In [117]:
%%time
# путь к справочнику МО
mo_path = "C:/personal_auto/mo.xlsx"

# загрузка справочника МО
df_mo = pd.read_excel(
    mo_path,
    sheet_name="Справочник",
    index_col=None,
    dtype="str",
)

# только актуальные данные МО
df_mo = df_mo[df_mo["deleteDate"].isnull()]

df_mo = df_mo[["oid", "moDeptName"]]
df_mo["moDeptName"] = df_mo["moDeptName"].fillna("не определено")

CPU times: total: 9.3 s
Wall time: 35.3 s


In [118]:
%%time
# путь к безликим
unpersonal_path = 'C:/personal_auto/unpersonal/'

# список файлов в целевой директории
source_files_list = os.listdir(unpersonal_path)

# загрузка безликих
df_dt = pd.DataFrame()
for file in source_files_list:
    try:
        df_tmp = pd.read_csv(
            unpersonal_path + file,
            sep=";",
            encoding="ansi",
        )
        df_dt = pd.concat([df_dt, df_tmp])
    except Exception:
        print(f"файл {file} имеет недопустимый тип и не был обработан")
df_dt = df_dt.replace(",", ".", regex=True).apply(pd.to_numeric, errors="ignore")
df_dt['3. Адрес МО. Наим. субъекта РФ'] = df_dt['3. Адрес МО. Наим. субъекта РФ'].fillna('Кемская волость')
# добавление поля согласованного названия региона
df_dt = df_dt.merge(df_reg_ref, left_on="3. Адрес МО. Наим. субъекта РФ", right_on="region_name_file", how="left")
df_dt.drop("region_name_file", axis=1, inplace=True)

# добавление поля ведомственной принадлежности МО
df_dt = df_dt.merge(df_mo, left_on="1. OID МО", right_on="oid", how="left")
df_dt.drop("oid", axis=1, inplace=True)

файл Readme.md имеет недопустимый тип и не был обработан
CPU times: total: 24 s
Wall time: 1min 9s


In [119]:
%%time
# путь к персонифицированным
personal_path = "C:/personal_auto/personal.xlsx"

# загрузка персонифицированных
df_dtp = pd.read_excel(
    personal_path,
    sheet_name="frmr_depart",
    index_col=None,
    dtype="str",
)

# преобразование типов
try:
    df_dtp["Дата рождения"] = pd.to_datetime(
        df_dtp["Дата рождения"], format="ISO8601"
    ).dt.date
except Exception as e:
    print("Проверьте формат даты рождения в исходном файле")
    print(e)
try:
    df_dtp["Дата начала"] = pd.to_datetime(
    df_dtp["Дата начала"], format="ISO8601"
    ).dt.date
except Exception as e:
    print("Проверьте формат даты начала в исходном файле")
    print(e)
try:
    df_dtp["Дата окончания"] = pd.to_datetime(
    df_dtp["Дата окончания"], format="ISO8601"
    ).dt.date
except Exception as e:
    print("Проверьте формат даты начала в исходном файле")
    print(e)
try:
    df_dtp["Дата начала неисполнения"] = pd.to_datetime(
    df_dtp["Дата начала неисполнения"], format="ISO8601"
    ).dt.date
except Exception as e:
    print("Проверьте формат даты начала неисполнения в исходном файле")
    print(e)
try:
    df_dtp["Дата окончания неисполнения"] = pd.to_datetime(
    df_dtp["Дата окончания неисполнения"], format="ISO8601"
    ).dt.date
except Exception as e:
    print("Проверьте формат даты окончания неисполнения в исходном файле")
    print(e)
try:
    df_dtp["Ставка"] = df_dtp["Ставка"].astype("float")
except Exception as e:
    print("Проверьте формат чисел в столбце 'Ставка'")
    print(e)

# добавление столбца уникальных
df_dtp["unique"] = df_dtp.apply(
    lambda row: str(row["Фамилия"]) + str(row["Имя"]) + str(row["Отчество"]) + str(row["Пол"]) + str(row["Дата рождения"]), axis=1
)

# добавление поля согласованного названия региона
df_dtp = df_dtp.merge(df_reg_ref, left_on="Субъект РФ", right_on="region_name_file", how="left")
df_dtp.drop("region_name_file", axis=1, inplace=True)

# добавление поля ведомственной принадлежности МО
df_dtp = df_dtp.merge(df_mo, left_on="OID организации", right_on="oid", how="left")
df_dtp.drop("oid", axis=1, inplace=True)

CPU times: total: 10.7 s
Wall time: 37.8 s


In [128]:
# список регионов из исходных файлов
df_dt_reg = pd.DataFrame(df_dt["3. Адрес МО. Наим. субъекта РФ"].unique())
df_dtp_reg = pd.DataFrame(df_dtp["Субъект РФ"].unique())
df_reg = (
    pd.concat([df_dt_reg, df_dtp_reg])
    .sort_values(by=0)
    .drop_duplicates()
    .reset_index(drop=True)
)
# если нужно выгрузить в файл
# df_reg.to_csv("C:/personal_auto/regs.csv", sep=";", encoding="utf-8-sig")

# проверка: все ли названия регионов из исходных файлах есть в таблице соответствия
df_regname_test = df_dt_reg.merge(
    df_reg_ref, left_on=0, right_on="region_name_file", how="left"
)
df_regname_test = df_regname_test[
    (df_regname_test["region_name_file"].isnull())
    | df_regname_test["real_region_name"].isnull()
]
if df_regname_test.shape[0] != 0:
    print(
        "Названия регионов есть в исходных файлах, но отсутствуют в таблице соответствия:"
    )
    print(df_regname_test.to_string(index=False))
else:
    print("Все названия регионов есть в справочнике")


# список специальностей из исходных файлов
df_dt_spec = pd.DataFrame(df_dt["24. Наим. долж-ти врача или ср. МП"].unique())
df_dtp_spec = pd.DataFrame(df_dtp["Должность"].unique())
df_spec = (
    pd.concat([df_dt_spec, df_dtp_spec])
    .sort_values(by=0)
    .drop_duplicates()
    .reset_index(drop=True)
    .dropna()
)
# если нужно выгрузить в файл
# df_spec.to_csv("C:/personal_auto/spec.csv", sep=";", encoding="utf-8-sig")

# проверка: все ли названия специальностей из исходных файлах есть в списке конфигурации
df_spec_test = df_spec.merge(
    df_spec_groups, left_on=0, right_on="spec_name", how="left"
)
df_spec_test = df_spec_test[df_spec_test["spec_name"].isnull()]
if df_spec_test.shape[0] != 0:
    print(
        "Названия специальностей есть в исходных файлах, но отсутствуют в файле конфигурации:"
    )
    print(df_spec_test.to_string(index=False))
else:
    print("Все названия специальностей есть в справочнике")


# список совместительства из файла персональных
df_cross = (
    pd.DataFrame(df_dtp["Тип занятия должности"].unique())
    .reset_index(drop=True)
    .dropna()
)
# если нужно выгрузить в файл
# df_cross.to_csv("C:/personal_auto/cross.csv", sep=";", encoding="utf-8-sig")

# проверка: все ли типы совместительства из исходных файлах есть в списке конфигурации
df_cross_test = df_cross.merge(
    df_cross_config, left_on=0, right_on="cross_type", how="left"
)
df_cross_test = df_cross_test[df_cross_test["cross_type"].isnull()]
if df_cross_test.shape[0] != 0:
    print(
        "Тип совместительства есть в исходных файлах, но отсутствуют в файле конфигурации:"
    )
    print(df_cross_test.to_string(index=False))
else:
    print("Все названия типов совместительства есть в справочнике")


# список временного неисполнения из файла персональных
df_tmp = (
    pd.DataFrame(
        df_dtp["Причина временного неисполнения функциональных обязанностей"].unique()
    )
    .reset_index(drop=True)
    .dropna()
)
# если нужно выгрузить в файл
# df_tmp.to_csv("C:/personal_auto/tmp.csv", sep=";", encoding="utf-8-sig")

# проверка: все ли названия специальностей из исходных файлах есть в списке конфигурации
df_tmp_test = df_tmp.merge(df_tmp_config, left_on=0, right_on="tmp_type", how="left")
df_tmp_test = df_tmp_test[df_tmp_test["tmp_type"].isnull()]
if df_tmp_test.shape[0] != 0:
    print(
        "Тип временного неисполнения есть в исходных файлах, но отсутствуют в файле конфигурации:"
    )
    print(df_tmp_test.to_string(index=False))
else:
    print("Все названия типов временного неисполнения есть в справочнике")

# список ведомственной принадлежности из файла конфигурации
df_dpt = pd.DataFrame(df_mo["moDeptName"].unique()).reset_index(drop=True).dropna()
# если нужно выгрузить в файл
# df_dpt.to_csv("C:/personal_auto/dpt.csv", sep=";", encoding="utf-8-sig")

# проверка: все ли названия типов ведомственной принадлежности из справочника МО есть в списке конфигурации
df_dpt_test = df_mo.merge(
    df_dpt_config, left_on="moDeptName", right_on="dpt_name", how="left"
)
df_dpt_test = df_dpt_test[df_dpt_test["dpt_name"].isnull()]
df_dpt_test.drop("oid", axis=1, inplace=True)
if df_dpt_test.shape[0] != 0:
    print(
        "Тип ведомственной принадлежности есть в справочнике МО, но отсутствуют в файле конфигурации:"
    )
    print(df_dpt_test.to_string(index=False))
else:
    print("Все типы ведомственной принадлежности есть в файле конфигурации")

Все названия регионов есть в справочнике
Все названия специальностей есть в справочнике
Все названия типов совместительства есть в справочнике
Все названия типов временного неисполнения есть в справочнике
Все типы ведомственной принадлежности есть в файле конфигурации


#### Фильтрация исходных файлов и получение результата

In [144]:
# фильтр по ведомственной принадлежности
if len(df_dpt_config_f["dpt_name"].tolist()) != 0:
    df_dt_f = df_dt[df_dt["moDeptName"].isin(df_dpt_config_f["dpt_name"].tolist())]
    df_dtp_f = df_dtp[df_dtp["moDeptName"].isin(df_dpt_config_f["dpt_name"].tolist())]
else:
    print(
        "Необходимо указать минимум один тип ведомственной принадлежности (вкладка dpt файла конфигурации)\nФильтарция не осуществлена"
    )
    df_dt_f = df_dt
    df_dtp_f = df_dtp

# фильтр по типу совместительства
if len(df_cross_config["cross_type"].tolist()) != 0:
    df_dtp_f = df_dtp_f[
        df_dtp_f["Тип занятия должности"].isin(df_cross_config["cross_type"].tolist())
    ]
else:
    print(
        "Необходимо указать минимум один тип выполнения обязанностей по занимаемой должности (вкладка cross файла конфигурации)\nФильтарция не осуществлена, данные могут быть неккоректными"
    )

# фильтр по временному неисполнению
if len(df_tmp_config_f["tmp_type"].tolist()) == 0:
    df_dtp_f = df_dtp_f[
        df_dtp_f["Причина временного неисполнения функциональных обязанностей"].isnull()
    ]
else:
    df_dtp_f = df_dtp_f[
        df_dtp_f["Причина временного неисполнения функциональных обязанностей"].isin(
            df_tmp_config_f["tmp_type"].tolist()
        )
    ]

# присоединение номеров групп специальностей
df_dt_f = df_dt_f.merge(
    df_spec_groups_f,
    left_on="24. Наим. долж-ти врача или ср. МП",
    right_on="spec_name",
    how="left",
)
df_dt_f.drop("spec_name", axis=1, inplace=True)
# только выбранные специальности:
df_dt_f_spec = df_dt_f[~df_dt_f["group"].isnull()]

df_dtp_f = df_dtp_f.merge(
    df_spec_groups_f,
    left_on="Должность",
    right_on="spec_name",
    how="left",
)
df_dtp_f.drop("spec_name", axis=1, inplace=True)
# только выбранные специальности:
df_dtp_f_spec = df_dtp_f[~df_dtp_f["group"].isnull()]

# количество уникальных персоналий по группам
df_dtp_result = pd.DataFrame(
    df_dtp_f_spec.groupby(["real_region_name", "group"])["unique"].count()
).reset_index()

# количество ставок имеющихся и занятых по безликим
df_dt_result = (
    df_dt_f_spec.groupby(["real_region_name", "group"])
    .sum()[
        [
            "25. Число долж-тей в целом по СП шт.",
            "26. Число долж-тей в целом по СП зан.",
        ]
    ]
    .reset_index()
)

# общий результат
df_result = (
    (
        df_dt_result.merge(
            df_dtp_result,
            left_on=["real_region_name", "group"],
            right_on=["real_region_name", "group"],
            how="outer",
        )
        .merge(df_spec_groups_merge, left_on="group", right_on="group", how="outer")
        .sort_values(by=["real_region_name", "group"])[
            [
                "real_region_name",
                "spec_name",
                "25. Число долж-тей в целом по СП шт.",
                "26. Число долж-тей в целом по СП зан.",
                "unique",
            ]
        ]
    )
    .fillna(
        {
            "unique": 0,
            "25. Число долж-тей в целом по СП шт.": 0,
            "26. Число долж-тей в целом по СП зан.": 0,
        }
    )
    .rename(
        columns={
            "unique": "individual",
            "25. Число долж-тей в целом по СП шт.": "rate_all",
            "26. Число долж-тей в целом по СП зан.": "rate_occupied",
        }
    )
).reset_index(drop=True)
df_result["individual"] = df_result["individual"].astype("int")

In [145]:
df_result.head(20)

,real_region_name,spec_name,rate_all,rate_occupied,individual
0,Алтайский край,врач общей практики (семейный врач),86.75,57.75,70
1,Алтайский край,"врач-психиатр детский, врач-психиатр детский у...",33.75,14.50,24
2,Алтайский край,"врач-психиатр-нарколог, врач-психиатр-нарколог...",187.25,113.25,139
3,Алтайский край,"врач - детский хирург, врач-хирург",507.75,390.95,168
4,Алтайский край,"врач-стоматолог, врач-стоматолог детский, врач...",431.50,319.75,374
5,Амурская область,врач общей практики (семейный врач),45.75,37.00,45
6,Амурская область,"врач-психиатр детский, врач-психиатр детский у...",9.50,6.00,5
7,Амурская область,"врач-психиатр-нарколог, врач-психиатр-нарколог...",50.75,45.50,57
8,Амурская область,"врач - детский хирург, врач-хирург",159.25,132.75,48
9,Амурская область,"врач-стоматолог, врач-стоматолог детский, врач...",79.50,65.25,70


***

***

## Дальше - автоматическая замена названий регионов, имеющих разное написание. Прикольно, но не надёжно.

#### Поиск и замена названий регионов, имеющих различные наименования

In [45]:
%%time
# список регионов для безликих
df_regions_u = pd.DataFrame(df_dt['3. Адрес МО. Наим. субъекта РФ']).drop_duplicates().reset_index(drop=True)

# таблица замены для обезличенных
df_crossregions_u = pd.merge(df_regions_u, df_regions_u, how="cross")
df_crossregions_u["real_region_name"] = df_crossregions_u.apply(
    lambda row: real_region_name(
        row["3. Адрес МО. Наим. субъекта РФ_x"], row["3. Адрес МО. Наим. субъекта РФ_y"]
    ),
    axis=1,
)
df_realregions_u = df_crossregions_u.reset_index(drop=True)
df_realregions_u = df_realregions_u.dropna().reset_index(drop=True)
df_realregions_u = df_realregions_u[
    df_realregions_u["3. Адрес МО. Наим. субъекта РФ_x"]
    != df_realregions_u["3. Адрес МО. Наим. субъекта РФ_y"]
]
# исключение из списка автозамены названий, не подвластных алгоритму
drop_regions_list_u = ["Республика Алтай",
                     "Ямало-Ненецкий автономный округ",
                     "Республика Саха (Якутия)",
                     "Алтайский край",
                     "Ненецкий автономный округ",
                     "Сахалинская область",
]
df_realregions_u = df_realregions_u.set_index("3. Адрес МО. Наим. субъекта РФ_x")
for region_name in drop_regions_list_u:
    df_realregions_u.drop(region_name, axis=0,inplace=True)
df_realregions_u.reset_index(drop=True , inplace=True )
df_realregions_u = df_realregions_u.rename(columns={"3. Адрес МО. Наим. субъекта РФ_y": "region_name"})
df_realregions_u = df_realregions_u[df_realregions_u["region_name"] != df_realregions_u["real_region_name"]]
df_realregions_u

CPU times: total: 46.9 ms
Wall time: 166 ms


,region_name,real_region_name
0,Кемеровская область,Кемеровская область - Кузбасс
2,Республика Адыгея,Республика Адыгея (Адыгея)
4,Республика Татарстан,Республика Татарстан (Татарстан)
6,Чувашская Республика,Чувашская Республика - Чувашия


In [6]:
%%time
# список регионов для персонифицированных
df_regions_p = pd.DataFrame(df_dtp['Субъект РФ']).drop_duplicates().reset_index(drop=True)

# таблица замены для персонифицированных
df_crossregions_p = pd.merge(df_regions_p, df_regions_p, how="cross")
df_crossregions_p["real_region_name"] = df_crossregions_p.apply(
    lambda row: real_region_name(
        row["Субъект РФ_x"], row["Субъект РФ_y"]
    ),
    axis=1,
)

df_realregions_p = df_crossregions_p.reset_index(drop=True)
df_realregions_p = df_realregions_p.dropna().reset_index(drop=True)
df_realregions_p = df_realregions_p[
    df_realregions_p["Субъект РФ_x"]
    != df_realregions_p["Субъект РФ_y"]
]

# исключение из списка автозамены названий, не подвластных алгоритму
drop_regions_list_p = ["Республика Алтай",
                     "Ямало-Ненецкий автономный округ",
                     "Республика Саха (Якутия)",
                     "Алтайский край",
                     "Ненецкий автономный округ",
                     "Сахалинская область",
]
df_realregions_p = df_realregions_p.set_index("Субъект РФ_x")
for region_name in drop_regions_list_p:
    df_realregions_p.drop(region_name, axis=0,inplace=True)
df_realregions_p.reset_index(drop=True , inplace=True )
df_realregions_p = df_realregions_p.rename(columns={"Субъект РФ_y": "region_name"})
df_realregions_p = df_realregions_p[df_realregions_p["region_name"] != df_realregions_p["real_region_name"]]
df_realregions_p

CPU times: total: 62.5 ms
Wall time: 96.8 ms


,region_name,real_region_name


##### Замена названий регионов

In [125]:
df_dt["real_region_name"] = df_dt.apply(lambda row: region_name_change (row["3. Адрес МО. Наим. субъекта РФ"], df_realregions_u), axis=1)
df_dtp["real_region_name"] = df_dtp.apply(lambda row: region_name_change (row["Субъект РФ"], df_realregions_p), axis=1)

In [35]:
word_2 = "Чечня"
word_1 = "Чеченская"
letter_full_part(word_1, word_2)

Чечня Чеченская
чеч


0.6

In [47]:
df_dtp

,СНИЛС,Фамилия,Имя,Отчество,Пол,Дата рождения,Субъект РФ,OID организации,Вид деятельности организации,Профиль деятельности организации,...,Тип занятия должности,Должность по фед справочнику,Должность,Ставка,Дата начала,Дата окончания,Причина временного неисполнения функциональных обязанностей,Дата начала неисполнения,Дата окончания неисполнения,unique
0,14097540061,Орлова,Ирина,Николаевна,Жен,1987-07-13,Республика Адыгея,1.2.643.5.1.13.13.12.2.1.14,Диспансер,противотуберкулезный,...,Основная,медицинский психолог,медицинский психолог,1.00,2009-06-10,NaN,NaN,NaN,NaN,ОрловаИринаНиколаевнаЖен1987-07-13
1,04267058754,Дзеньдзюк,Наталья,Валерьевна,Жен,1975-01-31,Республика Адыгея,1.2.643.5.1.13.13.12.2.1.14,Диспансер,противотуберкулезный,...,Совместительство внешнее,врач-психиатр-нарколог,врач-психиатр-нарколог,0.25,2021-08-02,NaN,NaN,NaN,NaN,ДзеньдзюкНатальяВалерьевнаЖен1975-01-31
2,05803960165,Ермакова,Инна,Анатольевна,Жен,1972-08-08,Республика Адыгея,1.2.643.5.1.13.13.12.2.1.15,Больница (в том числе детская),NaN,...,Совместительство внешнее,логопед,логопед,0.50,2023-03-06,NaN,NaN,NaN,NaN,ЕрмаковаИннаАнатольевнаЖен1972-08-08
3,06052967964,Христич,Елена,Никифоровна,Жен,1957-07-13,Республика Адыгея,1.2.643.5.1.13.13.12.2.1.15,Больница (в том числе детская),NaN,...,Совместительство внутреннее,медицинский психолог,медицинский психолог,0.25,2020-05-01,NaN,NaN,NaN,NaN,ХристичЕленаНикифоровнаЖен1957-07-13
4,04145752337,Васильева,Светлана,Алексеевна,Жен,1971-08-06,Республика Адыгея,1.2.643.5.1.13.13.12.2.1.15,Больница (в том числе детская),NaN,...,Основная,медицинский психолог,медицинский психолог,0.50,2022-08-01,NaN,NaN,NaN,NaN,ВасильеваСветланаАлексеевнаЖен1971-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42244,21807050435,Крыжова,Людмила,Флевияновна,Жен,1961-01-12,Луганская Народная Республика,1.2.643.5.1.13.13.12.2.94.80839,Больница (в том числе детская),NaN,...,Совместительство внутреннее,врач-психиатр участковый,врач-психиатр участковый,0.50,2023-01-09,NaN,NaN,NaN,NaN,КрыжоваЛюдмилаФлевияновнаЖен1961-01-12
42245,21807050435,Крыжова,Людмила,Флевияновна,Жен,1961-01-12,Луганская Народная Республика,1.2.643.5.1.13.13.12.2.94.80839,Больница (в том числе детская),NaN,...,Основная,врач-психиатр-нарколог,врач-психиатр-нарколог,1.00,2023-01-01,NaN,NaN,NaN,NaN,КрыжоваЛюдмилаФлевияновнаЖен1961-01-12
42246,21962927494,Радич,Александр,Григорьевич,Жен,1976-08-25,Луганская Народная Республика,1.2.643.5.1.13.13.12.2.94.80863,Больница (в том числе детская),NaN,...,Основная,врач-психиатр-нарколог,врач-психиатр-нарколог,1.00,2023-01-01,NaN,NaN,NaN,NaN,РадичАлександрГригорьевичЖен1976-08-25
42247,21816314040,Ференс,Игорь,Александрович,Муж,1956-04-29,Луганская Народная Республика,1.2.643.5.1.13.13.12.2.94.80869,Больница (в том числе детская),NaN,...,Совместительство внешнее,врач-психиатр участковый,врач-психиатр участковый,0.50,2022-12-01,NaN,NaN,NaN,NaN,ФеренсИгорьАлександровичМуж1956-04-29


In [6]:
str_0[0:3]

'Чув'

***

In [ ]:
# проверяем количество файлов в целевой директории
if len(source_file) == 0:
    print("Нет целевого файла, программа завершает работу")
    sys.exit()
elif len(source_file) > 1:
    print(f"В целевой директории {len(source_file)} файла(ов)")
    # тогда будем использовать файл с максимальным временем модификации
    # словарь для хранения времени модификации (ключ) файлов (значение)
    dict_ = {}
    # список времён модификации файлов
    list_ = []
    for file in source_file:
        path = soutce_path + file
        mtime = os.path.getmtime(path)
        dict_[mtime] = file
        list_.append(mtime)
    list_.sort(reverse=True)
    source_file_result = dict_[list_[0]]
    print(
        f"Для работы используется файл {source_file_result}, имеющий наиболее позднее время модификации\nЕсли это не то, что вам нужно, измените содержимое целевой директории"
    )
else:
    source_file_result = source_file[0]
    print(f"В целевой директории единственный файл {source_file_result}")

# путь к целевому файлу
soutce_file_path = soutce_path + source_file_result

# префикс для имён файлов-результатов
pref = source_file_result.split(".")[0]

# полный путь к итоговому экселевскому файлу ФЛК-1,2
result_12_exc_path = result_12_exc_path + pref + "_flk12_result.xlsx"
# полный путь к итоговому файлу ФЛК-3
result_3_path = result_3_path + pref + "_flk3_result.csv"
# полный путь к архиву
result_12_files_path_arch = result_12_files_path_arch + pref + "_flk12_result"


# ФЛК-1, 2

# данные для фильтрации исходного файла
last_status = [
    "На согласовании",
    "На редактировании (после отклонения)",
    "Запрос на отклонение",
    "Согласовано",
]
# last_status = ["На согласовании", "На редактировании (после отклонения)"]
last_status_date = "2020-07-10"  # сначала год, месяц, потом число. Отчётный период начинается в 00:00 указанной даты
next_status_date = "2073-07-03"  # и продолжается до 23:59 даты перед конечной

print(f"Статус: {last_status}")
print(f"Дата начала: {last_status_date}")
print(f"Дата окончания: {next_status_date}")

# сделать пустые фреймы для всех фреймов ошибок для корректной выгруки в Excel

# удаление всех фреймов, оставшихся (вдруг) от предыдущей работы скрипта
try:
    del df_notation
except Exception:
    pass
try:
    del df_ass_0
except Exception:
    pass
try:
    del df_ass
except Exception:
    pass
try:
    del df_ass_success
except Exception:
    pass
try:
    del df_ass_error
except Exception:
    pass
try:
    del df_mo_error
except Exception:
    pass
try:
    del df_dept_error
except Exception:
    pass
try:
    del df_ass_erorr_sum
except Exception:
    pass
try:
    del df_mo_count_more_save
except Exception:
    pass
try:
    del df_ass_mo_plan_zero
except Exception:
    pass
try:
    del df_ass_mo_fact_zero
except Exception:
    pass


# пустые фреймы для будущих ошибок (без них вылетает сохранение в excel, когда нет ошибок)
df_dept_error = pd.DataFrame(
    columns=[
        "id СП",
        "Регион",
        "наименование оборудования",
        "непосредственно заявка СП",
        "сумма заявок по кабинетам",
    ]
)
df_mo_error = pd.DataFrame(
    columns=[
        "id МО",
        "Регион",
        "наименование оборудования",
        "непосредственно заявка МО",
        "сумма заявок по СП",
    ]
)
df_ass_mo_plan_zero = pd.DataFrame()
df_ass_mo_fact_zero = pd.DataFrame()

# загрузка исходного файла - путь изменён!
df_ass = pd.read_csv(
    soutce_file_path,
    sep=";",
    header=1,
    encoding="utf-8-sig",
)

# проверка наличия новых столбцов
if (
    df_ass.columns[1] != "Текущий статус"
    or df_ass.columns[2] != "Дата последнего статуса"
):
    print("Неверная структура исходных данных")
else:
    print("Структура исходных данных верная")

# преобразование формата даты для фильтрации
# df_ass["Дата последнего статуса"] = df_ass["Дата последнего статуса"].astype("datetime64[ns]", format = "%d.%m.%Y %H:%M:%S")
df_ass["Дата последнего статуса"] = pd.to_datetime(
    df_ass["Дата последнего статуса"], format="%d.%m.%Y %H:%M:%S"
)

# копия исходника для потомков
df_ass_0 = df_ass.copy()

# фильтрация по статусу и последней дате, удаление столбцов статуса и последней даты
df_ass = df_ass[df_ass["Текущий статус"].isin(last_status)]
df_ass = df_ass[df_ass["Дата последнего статуса"] >= last_status_date]
df_ass = df_ass[df_ass["Дата последнего статуса"] < next_status_date]
df_ass = df_ass.reset_index(drop=True)
# сначала сохраним в файл для контроля
df_ass.to_csv(
    result_12_files_path + "/исходные_данные_до_удаления_столбцов.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)

cols = [1, 2]
df_ass = df_ass.drop(df_ass.columns[cols], axis=1)
# теперь получаем нормальный исходник, с которым работали до с самого начала, а не вот это вот всё
# дальше работаем так, словно ничего в исходнике не менялось


# очистка полностю пустых строк
row_count_0 = df_ass.shape[0]
df_ass = df_ass.dropna(how="all")
print(f"Удалено {row_count_0 - df_ass.shape[0]} полностью пустых строк")

# заполнение пустых значений вниз
df_ass[df_ass.columns[0]] = df_ass[[df_ass.columns[0]]].ffill(axis=0)
df_ass[df_ass.columns[6]] = df_ass[[df_ass.columns[6]]].ffill(axis=0)
df_ass[df_ass.columns[7]] = df_ass[[df_ass.columns[7]]].ffill(axis=0)

# замена пропусков на 0
df_ass = df_ass.fillna(0)

# сохранение результата в файл
df_ass.to_csv(
    result_12_files_path + "/исходные_данные(.csv).csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)

# добавление столбца уровня лечебницы
df_ass = df_ass.astype({df_ass.columns[1]: "str"})
df_ass["lvl"] = df_ass.apply(lambda x: len(x[df_ass.columns[1]].split("_")) - 2, axis=1)

# переименование столбца с некошерным названием
df_ass = df_ass.rename(columns={df_ass.columns[1]: "ID"})

# разделение на фреймы МО, СП и кабинетов, удаление ненужных данных
df_ass_mo = df_ass[df_ass["lvl"] == 0].reset_index(drop=True)
df_ass_dept = df_ass[df_ass["lvl"] == 1].reset_index(drop=True)
df_ass_room = df_ass[df_ass["lvl"] == 2].reset_index(drop=True)
df_ass_mo.drop(
    [
        df_ass.columns[0],
        df_ass.columns[2],
        df_ass.columns[3],
        df_ass.columns[4],
        df_ass.columns[5],
        df_ass.columns[6],
        df_ass.columns[7],
        df_ass.columns[8],
        "lvl",
    ],
    axis=1,
    inplace=True,
)
df_ass_dept.drop(
    [
        df_ass.columns[0],
        df_ass.columns[2],
        df_ass.columns[3],
        df_ass.columns[4],
        df_ass.columns[5],
        df_ass.columns[6],
        df_ass.columns[7],
        df_ass.columns[8],
        "lvl",
    ],
    axis=1,
    inplace=True,
)
df_ass_room.drop(
    [
        df_ass.columns[0],
        df_ass.columns[2],
        df_ass.columns[3],
        df_ass.columns[4],
        df_ass.columns[5],
        df_ass.columns[6],
        df_ass.columns[7],
        df_ass.columns[8],
        "lvl",
    ],
    axis=1,
    inplace=True,
)

# а что, если в фрейме МО какие-то лечебницы встречаются более одного раза?
df_mo_count = pd.DataFrame(df_ass_mo["ID"].value_counts()).reset_index()
df_mo_count_more = df_mo_count[df_mo_count["count"] > 1]
df_mo_count_more_save = df_mo_count_more.merge(
    df_ass, left_on="ID", right_on="ID", how="left"
)  # для сохранения
# удалим из сохраняемого фрейма столбцы, заполненные в данном случае - при повторяющихся кодах лечебниц - некорректно
df_mo_count_more_save = df_mo_count_more_save.drop(
    [
        "Регион",
        "Численность прикрепленного населения",
        "Численность обслуживаемого населения ",
    ],
    axis=1,
)
if len(df_mo_count_more["ID"].tolist()) > 0:
    print("Внимание! В исходных данных повторяются следующие МО:")
    print(*df_mo_count_more["ID"].tolist(), sep="\n")
else:
    print("Коды МО в исходных данных не повторяются")

# а что ести какие-то фреймы пусты?
flag = 0  # показатель уровня иерархии
if df_ass_dept.empty:
    flag = 1
    print("Иерархия данных отсутствует, представлены только МО")
if df_ass_room.empty:
    flag = 2
    print("Представлены данные по МО и подразделениям, кабинетов нет")


if flag == 0:  # есть и МО, и СП, и кабинеты
    print("Структура данных полная, есть и МО, и СП, и кабинеты")
    # добавление в фреймы СП и кабинетов id более выского уровня иерархии
    df_ass_dept.insert(
        1, "id", df_ass_dept.apply(lambda x: "_".join(x["ID"].split("_")[:-1]), axis=1)
    )
    df_ass_room.insert(
        1, "id", df_ass_room.apply(lambda x: "_".join(x["ID"].split("_")[:-1]), axis=1)
    )

    # группировка фреймов СП и кабинетов по более высокому уровню иерархии
    df_ass_dept_group_mo = df_ass_dept.groupby("id").sum().reset_index()
    df_ass_dept_group_room = df_ass_dept.groupby("ID").sum().reset_index()
    df_ass_room_group = df_ass_room.groupby("id").sum().reset_index()
    df_ass_dept_group_mo.drop(["ID"], axis=1, inplace=True)
    df_ass_dept_group_room.drop(["id"], axis=1, inplace=True)
    df_ass_room_group.drop(["ID"], axis=1, inplace=True)

    # добавление данных об источнике
    df_ass_mo.insert(1, "source", "mo")
    df_ass_mo = df_ass_mo.rename(columns={"ID": "id"})
    df_ass_dept_group_mo.insert(1, "source", "dept")
    df_ass_dept_group_room.insert(1, "source", "dept")
    df_ass_dept_group_room = df_ass_dept_group_room.rename(columns={"ID": "id"})
    df_ass_room_group.insert(1, "source", "room")

    # соединение фреймов
    df_mo = (
        pd.concat([df_ass_mo, df_ass_dept_group_mo])
        .sort_values(["id"], ascending=False)
        .reset_index(drop=True)
    )
    df_dept = (
        pd.concat([df_ass_dept_group_room, df_ass_room_group])
        .sort_values(["id"], ascending=False)
        .reset_index(drop=True)
    )

    # формирование спиков кодов лечебниц строк с повторяющимися кодами лечебниц
    mo_index_list = []
    for index in range(0, df_mo.shape[0] - 1):
        if df_mo.loc[index]["id"] == df_mo.loc[index + 1]["id"]:
            mo_index_list.append(df_mo.loc[index]["id"])

    dept_index_list = []
    for index in range(0, df_dept.shape[0] - 1):
        if df_dept.loc[index]["id"] == df_dept.loc[index + 1]["id"]:
            dept_index_list.append(df_dept.loc[index]["id"])

    # удаление непарных строк
    df_mo_result = df_mo.loc[df_mo["id"].isin(mo_index_list)]
    df_dept_result = df_dept.loc[df_dept["id"].isin(dept_index_list)]
    for column in df_mo_result.columns:
        if df_mo_result[column].sum() == 0:
            df_mo_result = df_mo_result.drop(column, axis=1)
    df_mo_result = df_mo_result.reset_index(drop=True)

    for column in df_dept_result.columns:
        if df_dept_result[column].sum() == 0:
            df_dept_result = df_dept_result.drop(column, axis=1)
    df_dept_result = df_dept_result.reset_index(drop=True)

elif flag == 2:  # нет кабинетов, только МО и СП
    df_dept_result = pd.DataFrame()  # пустой фрейм для единообразия
    df_ass_dept.insert(
        1, "id", df_ass_dept.apply(lambda x: "_".join(x["ID"].split("_")[:-1]), axis=1)
    )

    # группировка фреймов СП и кабинетов по более высокому уровню иерархии
    df_ass_dept_group_mo = df_ass_dept.groupby("id").sum().reset_index()
    df_ass_dept_group_mo.drop(["ID"], axis=1, inplace=True)

    # добавление данных об источнике
    df_ass_mo.insert(1, "source", "mo")
    df_ass_mo = df_ass_mo.rename(columns={"ID": "id"})
    df_ass_dept_group_mo.insert(1, "source", "dept")

    # соединение фреймов
    df_mo = (
        pd.concat([df_ass_mo, df_ass_dept_group_mo])
        .sort_values(["id"], ascending=False)
        .reset_index(drop=True)
    )

    # формирование списков кодов лечебниц строк с повторяющимися кодами лечебниц
    mo_index_list = []
    for index in range(0, df_mo.shape[0] - 1):
        if df_mo.loc[index]["id"] == df_mo.loc[index + 1]["id"]:
            mo_index_list.append(df_mo.loc[index]["id"])

    # удаление непарных строк
    df_mo_result = df_mo.loc[df_mo["id"].isin(mo_index_list)]
    for column in df_mo_result.columns:
        if df_mo_result[column].sum() == 0:
            df_mo_result = df_mo_result.drop(column, axis=1)
    df_mo_result = df_mo_result.reset_index(drop=True)
else:
    print("Иерархия отсутстствует, нет предмета для проверки")

# удалим из исходных данных не нужный уже и вредный по сути столбец lvl
df_ass = df_ass.drop("lvl", axis=1)

# собственно проверка соответствия количества оборудования

# сделаем маленький кусочек исходного фрейма для присоединения к результатам
df_ass_l = df_ass[["Регион", "ID", "Наименование МО"]]

# проверка df_mo_result
mo_error = []
for mo in df_mo_result["id"].unique():
    df_test_mo = df_mo_result[df_mo_result["id"] == mo]
    # проверка на количество строк в df_test_mo
    # если строк не 2, вся информация сохраняется в файл и проверка такой МО не осуществляется
    if df_test_mo.shape[0] != 2 or len(df_test_mo["source"].unique()) != 2:
        path = result_12_files_path + "/" + mo + ".csv"
        df_test_mo.to_csv(path, sep=";", encoding="utf-8-sig", index=False)
    else:
        for column in df_test_mo.columns[2:]:
            if (
                df_test_mo.loc[df_test_mo["source"] == "mo"][column].values[0]
                < df_test_mo.loc[df_test_mo["source"] == "dept"][column].values[0]
            ):
                mo_error.append(mo)
                mo_error.append(column)
                mo_error.append(
                    df_test_mo.loc[df_test_mo["source"] == "mo"][column].values[0]
                )
                mo_error.append(
                    df_test_mo.loc[df_test_mo["source"] == "dept"][column].values[0]
                )
mo_error_list = [mo_error[i : i + 4] for i in range(0, len(mo_error), 4)]
df_mo_error = pd.DataFrame(
    mo_error_list,
    columns=[
        "id МО",
        "наименование оборудования",
        "непосредственно заявка МО",
        "сумма заявок по СП",
    ],
)

# присоединение данных из исходного фрейма
try:
    df_mo_error = df_mo_error.merge(
        df_ass_l, left_on="id МО", right_on="ID", how="left"
    )
    df_mo_error = df_mo_error[
        [
            "id МО",
            "Регион",
            "Наименование МО",
            "наименование оборудования",
            "непосредственно заявка МО",
            "сумма заявок по СП",
        ]
    ]
except Exception:
    print("Трабл при присоединении названия региона в ошибках МО")

# сохранение результата в файл
if df_mo_error.empty:
    print("Расхождений данных между МО и СП нет")
    df_mo_error = pd.DataFrame(
        columns=[
            "id МО",
            "наименование оборудования",
            "непосредственно заявка МО",
            "сумма заявок по СП",
        ]
    )
else:
    df_mo_error.to_csv(
        result_12_files_path + "/несоответствие_МО-СП.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Несоответствие между МО и СП сохранены в файл")

# проверка df_dept_result
if df_dept_result.empty:
    pass
else:
    dept_error = []
    for mo in df_dept_result["id"].unique():
        df_test_dept = df_dept_result[df_dept_result["id"] == mo]
        # проверка на количество строк в df_test_dept
        # если строк не 2, вся информация сохраняется в файл и проверка такого подразделения не осуществляется
        if df_test_dept.shape[0] != 2 or len(df_test_dept["source"].unique()) != 2:
            path = result_12_files_path + "/" + mo + "_dept.csv"
            df_test_dept.to_csv(path, sep=";", encoding="utf-8-sig", index=False)
        else:
            for column in df_test_dept.columns[2:]:
                if (
                    df_test_dept.loc[df_test_dept["source"] == "room"][column].values[0]
                    > df_test_dept.loc[df_test_dept["source"] == "dept"][column].values[
                        0
                    ]
                ):
                    dept_error.append(mo)
                    dept_error.append(column)
                    dept_error.append(
                        df_test_dept.loc[df_test_dept["source"] == "dept"][
                            column
                        ].values[0]
                    )
                    dept_error.append(
                        df_test_dept.loc[df_test_dept["source"] == "room"][
                            column
                        ].values[0]
                    )
                    # print(mo, column, df_test_dept.loc[df_test_dept['source']=='room'][column].values[0], df_test_dept.loc[df_test_dept['source']=='dept'][column].values[0])
    dept_error_list = [dept_error[i : i + 4] for i in range(0, len(dept_error), 4)]
    df_dept_error = pd.DataFrame(
        dept_error_list,
        columns=[
            "id СП",
            "наименование оборудования",
            "непосредственно заявка СП",
            "сумма заявок по кабинетам",
        ],
    )
    try:
        df_dept_error = df_dept_error.merge(
            df_ass_l, left_on="id СП", right_on="ID", how="left"
        )
        df_dept_error = df_dept_error[
            [
                "id СП",
                "Регион",
                "Наименование МО",
                "наименование оборудования",
                "непосредственно заявка СП",
                "сумма заявок по кабинетам",
            ]
        ]
    except Exception:
        print("Трабл при присоединении названия региона в ошибках СП")

    # сохранение результата в файл
    if df_dept_error.empty:
        df_dept_error = pd.DataFrame(
            columns=[
                "id СП",
                "наименование оборудования",
                "непосредственно заявка СП",
                "сумма заявок по кабинетам",
            ]
        )
    else:
        df_dept_error.to_csv(
            result_12_files_path + "/несоответствие_СП-кабинеты.csv",
            sep=";",
            encoding="utf-8-sig",
            index=False,
        )
        print("Несоответствия между СП и кабинетами сохранены в файл")

# годная для дальнейшей работы структура фреймов с ошибками, если ошибок не было (котстыль)
if df_dept_error.shape[0] == 0:
    df_dept_error = pd.DataFrame(
        columns=[
            "id СП",
            "Регион",
            "наименование оборудования",
            "непосредственно заявка СП",
            "сумма заявок по кабинетам",
        ]
    )
if df_mo_error.shape[0] == 0:
    df_mo_error = pd.DataFrame(
        columns=[
            "id МО",
            "Регион",
            "наименование оборудования",
            "непосредственно заявка МО",
            "сумма заявок по СП",
        ]
    )

# проверка на заполненность данных - ищем регионы, которые не попросили вообще ничего
# группировка и суммирование количества оборудования по регионам
cols = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
]  # столбцы для удаления (иначе трабл с суммированием по ненужным столбцам, где есть и буквы, и цифры)
df_ass_regions = df_ass.drop(df_ass.columns[cols], axis=1)
df_ass_regions_sum = df_ass_regions.groupby("Регион").sum().reset_index()
# добавляем столбец с суммой заявок по регионам
df_ass_regions_sum["region_sum"] = df_ass_regions_sum[list(df_ass_regions_sum)[1:]].sum(
    axis=1
)
# отбираем успешные и плохие регионы по сумме заявленного и формируем списки регионов
df_ass_success_sum = df_ass_regions_sum[
    df_ass_regions_sum["region_sum"] != 0
].reset_index()
df_ass_erorr_sum = df_ass_regions_sum[
    df_ass_regions_sum["region_sum"] == 0
].reset_index()
bad_regions_sum_set = set(df_ass_erorr_sum["Регион"].tolist())
bad_regions_sum_list = list(bad_regions_sum_set)
df_ass_erorr_sum = df_ass.loc[df_ass["Регион"].isin(bad_regions_sum_list)]

# проверка на заполненность данных 2 - ищем регионы с нулевым планом или нулевым фактом
# удаляем непотребные столбцы
cols = [
    2,
    3,
    4,
    5,
    6,
    7,
    8,
]  # столбцы для удаления (иначе трабл с суммированием по ненужным столбцам, где есть и буквы, и цифры)
df_ass_mo_regions = df_ass.drop(df_ass.columns[cols], axis=1)
# разделяем строки на "план" и "факт"
# удаляем столбцы "по факту"
del_columns_list_fact = []
for col in df_ass_mo_regions.columns:
    if col.split()[-1] == "факту":
        del_columns_list_fact.append(col)
df_ass_mo_plan = df_ass_mo_regions.drop(del_columns_list_fact, axis=1)
# удаляем столбцы "по плану"
del_columns_list_plan = []
for col in df_ass_mo_regions.columns:
    if col.split()[-1] == "плану":
        del_columns_list_plan.append(col)
df_ass_mo_fact = df_ass_mo_regions.drop(del_columns_list_plan, axis=1)
# добавим столбцы с суммой плана и факта по каждой строке
df_ass_mo_plan["count"] = df_ass_mo_plan.iloc[:, 2:].sum(axis=1)
df_ass_mo_fact["count"] = df_ass_mo_fact.iloc[:, 2:].sum(axis=1)
# отберём регионы с нулевым планом и нулевым фактом
df_ass_mo_plan_zero = df_ass_mo_plan[df_ass_mo_plan["count"] == 0]
df_ass_mo_fact_zero = df_ass_mo_fact[df_ass_mo_fact["count"] == 0]
# плохие регионы по сумме плана
bad_regions_sum_plan_set = set(df_ass_mo_plan_zero["Регион"].tolist())
bad_regions_sum_plan_list = list(bad_regions_sum_plan_set)
# плохие регионы по сумме факта
bad_regions_sum_fact_set = set(df_ass_mo_fact_zero["Регион"].tolist())
bad_regions_sum_fact_list = list(bad_regions_sum_set)
# если есть плохие регионы по сумме плана и факта, выгрузим их в файл
if df_ass_mo_plan_zero.empty:
    print("Регионов с нулевым планом нет")
    bad_regions_sum_plan_set = set()
else:
    df_ass_mo_plan_zero.to_csv(
        result_12_files_path + "/пустая_сумма_плана.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Данные по регионам с нулевой суммой плана сохранены в файл")
if df_ass_mo_fact_zero.empty:
    print("Регионов с нулевым фактом нет")
    bad_regions_sum_fact_set = set()
else:
    df_ass_mo_fact_zero.to_csv(
        result_12_files_path + "/пустая_сумма_факта.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Данные по регионам с нулевой суммой факта сохранены в файл")

# Антон велел убрать из успешных регионы с пустым планом ИЛИ пустой суммой. Но я это делать пока не буду, просто сохраню такие регионы
bad_regions_plan_fact_set = bad_regions_sum_fact_set.union(
    bad_regions_sum_plan_set
)  # множество плохих регионов по нулевому плану ИЛИ нулевому факту - нигде не используется!

# если плохие регионы есть, выгрузим их в файл
if df_ass_erorr_sum.empty:
    print("Регионов с полностью пустой заявкой нет")
else:
    df_ass_erorr_sum.to_csv(
        result_12_files_path + "/пустая_сумма_заявок.csv",
        sep=";",
        encoding="utf-8-sig",
        index=False,
    )
    print("Данные по регионам с нулевой суммой сохранены в файл")

# считаем ошибкой заявки ТОЛЬКО ПУСТОЙ ПЛАН: bad_regions_sum_plan_set
# удаление из исходных данных проблемных регионов (и тех, у которых суммы не бьются, и тех, кто план не заполнил) и сохранение результата в файл
try:
    bad_mo_region_set = set(df_mo_error["Регион"].tolist())
except Exception:
    bad_mo_region_set = set()
    print("Ошибок МО нет")
try:
    bad_dept_region_set = set(df_dept_error["Регион"].tolist())
except Exception:
    bad_dept_region_set = set()
    print("Ошибок СП нет")

# обьединяем все списики ошибочных регионов: ошибки сумм МО, ошибки сумм СП, регионы с нулевым планом
bad_regions_list = list(
    bad_mo_region_set.union(bad_dept_region_set).union(bad_regions_sum_plan_set)
)
df_ass_success = df_ass.loc[
    ~df_ass["Регион"].isin(bad_regions_list)
]  # успешные регионы - которых нет в списке ошибочных
df_ass_error = df_ass.loc[
    df_ass["Регион"].isin(bad_regions_list)
]  # плохие регионы, которые есть в списке ошибочных
df_ass_success.to_csv(
    result_12_files_path + "/успешные_регионы.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)
print("Данные по успешным регионам сохранены в файл")
df_ass_error.to_csv(
    result_12_files_path + "/регионы_с_ошибками.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
)
print("Данные по регионам с ошибками сохранены в файл")

# архивирование
zip_path = shutil.make_archive(result_12_files_path_arch, "zip", result_12_files_path)
# удалнение папки с файлами ФЛК-1,2
shutil.rmtree(result_12_files_path)
print(
    f"Архив с файлами сохранён: {zip_path}, временная папка {result_12_files_path} удалена"
)

# запись всего зоопарка в этот ваш excel

# разъяснения для юзера
df_notation = pd.DataFrame(
    {
        "данные": [
            "текущий статус",
            "дата с... (включая)",
            "дата по... (исключая)",
            "исходник",
            "фильтр дата-статус",
            "регионы без ошибок",
            "регионы с ошибками",
            "несоответствие МО-СП",
            "несоответствие СП-каб",
            "пустая заявка",
            "пустой план (ОШИБКА)",
            "пустой факт (НЕ ошибка)",
            "повторяющиеся МО",
        ],
        "содержание": [
            last_status,
            last_status_date,
            next_status_date,
            "исходные данные как они есть",
            "исходные данные после применения фильтра по текущему статусу и дате последнего статуса",
            "регионы, данные которых не содержат ошибок",
            "регионы, в данных которых выявлены ошибки",
            "данные по регионам, для которых количество заявленного оборудования в разрезе вида оборудования для МО мнее, чем суммарное количество заявленного оборудования в разрезе вида оборудования для СП, входящих в эту МО",
            "аналогичные данные для несоответствия между количеством оборудования, заявленного для СП, и суммарным количеством оборудования, заявленного для кабинетов, входящих в это СП",
            "данные по регионам, которые не указали вообще ничего (если есть хоть что-нибудь в любой позиции, регион сюда не попадёт) - это НЕ ошибка",
            "данные по регионам, у которых в позициях плана ничего не заявлено - это ОШИБКА",
            "данные по регионам, у которых в позициях факта ничего не заявлено - это НЕ ошибка",
            "данные по МО, которые упоминаются в исходной выгрузке более одного раза (такого быть не должно). Данные по таким МО не обрабатываются, регион не попадает в ошибочные и что сним делать, должен решать аналитик",
        ],
    }
)

# создание Pandas Excel writer, используя XlsxWriter в качестве движка
writer = pd.ExcelWriter(result_12_exc_path, engine="xlsxwriter")

# write each DataFrame to a specific sheet
df_notation.to_excel(writer, sheet_name="о чём этот файл", index=False)
df_ass_0.to_excel(writer, sheet_name="исходник", index=False)
df_ass.to_excel(writer, sheet_name="фильтр дата-статус", index=False)
df_ass_success.to_excel(writer, sheet_name="регионы без ошибок", index=False)
df_ass_error.to_excel(writer, sheet_name="регионы с ошибками", index=False)
df_mo_error.to_excel(writer, sheet_name="несоответствие МО-СП", index=False)
df_dept_error.to_excel(writer, sheet_name="несоответствие СП-каб", index=False)
df_ass_erorr_sum.to_excel(writer, sheet_name="пустая заявка (не ошибка)", index=False)
df_ass_mo_plan_zero.to_excel(writer, sheet_name="пустой план (ошибка)", index=False)
df_ass_mo_fact_zero.to_excel(writer, sheet_name="пустой факт (не ошибка)", index=False)
df_mo_count_more_save.to_excel(writer, sheet_name="повторяющиеся МО", index=False)

# close the Pandas Excel writer and output the Excel file
writer.close()
print("Данные сохранены в файл Excel")